In [1]:
from __future__ import division, print_function, absolute_import, unicode_literals
import numpy as np

import pygsti
from pygsti.construction import std1Q_XYI as std
from pygsti.construction import std2Q_XYICNOT as std2Q
from pygsti.extras import circuit
from pygsti.extras import newrb as prb

/Users/tjproct/pyGSTi/packages/pygsti/tools/matrixtools.py:23: UserWarning: Could not import Cython extension - falling back to slower pure-python routines
  _warnings.warn("Could not import Cython extension - falling back to slower pure-python routines")


In [2]:
# A list of standard hard-coded gate labels
gllist = ['Gi','Gh','Gp','Gcnot'] 

# The number of qubits
n = 4

# The availiability of gates that are not available to all qubit / qubit pairs
availability = {}

a = [(i,i+1) for i in range(0,n-1)] + [(n-1,0)]
availability={'Gcnot':a}

pspec = pygsti.obj.ProcessorSpec(n, gllist, availability=availability, verbosity=0)

In [4]:
sectors = [[pygsti.objects.Label('Gcnot',(0,1))],[pygsti.objects.Label('Gcnot',(1,2))]]

In [5]:
#c = prb.sample_crb_circuit(pspec,1)

In [6]:
c1, c2, c3 = prb.sample_prb_circuit(pspec,10,sampler='sectors',sampler_args={'sectors':sectors,'two_qubit_prob':0.5},return_partitioned=True)

In [7]:
print(c2)

Qubit 0 ---|Gi |-|Gh |-|Gi |-|Gh |-Gcnot:0:1|-|Gcnot:0:1|-|Gcnot:0:1|-||Gh |-|Gp |-Gcnot:0:1|-|--
Qubit 1 ---|Gp |-|Gi |-|Gh |-Gcnot:1:2|-|Gcnot:0:1|-|Gcnot:0:1|-|Gcnot:0:1|-||Gi |-Gcnot:1:2|-|Gcnot:0:1|-|--
Qubit 2 ---|Gh |-|Gh |-|Gh |-Gcnot:1:2|-||Gp |-|Gp |-|Gh |-|Gi |-Gcnot:1:2|-||Gp |---
Qubit 3 ---|Gp |-|Gp |-|Gp |-|Gp |-|Gi |-|Gi |-|Gi |-|Gi |-|Gi |-|Gh |---



In [37]:
c1, c2, c3 = prb.sample_prb_circuit(pspec,10,sampler='weights',sampler_args={'two_qubit_weighting':0.25},
                                    return_partitioned=True)

In [38]:
print(c2)

Qubit 0 ---|Gp |-Gcnot:0:1|-|Gcnot:3:0|-|Gcnot:0:1|-||Gp |-Gcnot:3:0|-||Gp |-Gcnot:3:0|-|Gcnot:3:0|-||Gp |---
Qubit 1 ---Gcnot:1:2|-|Gcnot:0:1|-||Gh |-Gcnot:0:1|-||Gp |-Gcnot:1:2|-||Gi |-|Gh |-|Gh |-|Gh |---
Qubit 2 ---Gcnot:1:2|-||Gi |-|Gh |-|Gp |-|Gh |-Gcnot:1:2|-|Gcnot:2:3|-||Gh |-|Gi |-Gcnot:2:3|-|--
Qubit 3 ---|Gi |-|Gh |-Gcnot:3:0|-||Gi |-|Gi |-Gcnot:3:0|-|Gcnot:2:3|-|Gcnot:3:0|-|Gcnot:3:0|-|Gcnot:2:3|-|--



## Test Labels

In [14]:
print(list(map(str,std.gs_target.gates.keys())))
print(std.gs_target.gates.keys())

['Gi', 'Gx', 'Gy']
odict_keys([Label('Gi',), Label('Gx',), Label('Gy',)])


In [ ]:
print(std.gs_target['Gx'])

In [ ]:
std.gs_target.probs(('Gx','Gy'))

## Test Circuit

In [ ]:
ps.models['clifford']['Gh',0].embedded_gate

In [ ]:
y = x.embedded_gate

In [ ]:
y.smatrix

In [ ]:
c = pygsti.obj.Circuit(gatestring=('Gx','Gy'), num_lines=1)

In [ ]:
print(c.line_items)
print(c.tup)
print(c.str)
print(c)

In [ ]:
from pygsti.objects import Label as L
gs = pygsti.obj.GateString( (L('Gx',0),L('Gy',1),L('Gcnot',(1,2)) ))
c2 = pygsti.obj.Circuit(gatestring=gs, num_lines=3)

In [ ]:
print(c2.line_items)
print(c2.tup)
print(c2.str)
print(c2)

In [ ]:
c3 = pygsti.obj.Circuit(gatestring=[ ('Gx','Q0'), ('Gy','Q1'), ('Gcnot','Q1','Q2') ], line_labels=('Q0','Q1','Q2'))
print(c3.line_items)
print(c3.tup)
print(c3.str)
print(c3)

In [ ]:
c3.done_editing()

### Probabilities from circuit (simulation)

In [ ]:
gs1 = std.gs_target.copy()
print(gs1.probs(c))
print(c.simulate(gs1))

In [ ]:
from pygsti.objects.gatemapcalc import GateMapCalc                                                                                                                                   
nQubits = 3
basis1Q = pygsti.obj.Basis("pp",2)
v0 = pygsti.construction.basis_build_vector("0", basis1Q)
v1 = pygsti.construction.basis_build_vector("1", basis1Q)

gs3 = pygsti.obj.GateSet()
gs3.stateSpaceLabels = ('Q0','Q1','Q2')
gs3.preps['rho0'] = pygsti.obj.TensorProdSPAMVec('prep',
    [pygsti.obj.TPParameterizedSPAMVec(v0) for i in range(nQubits)] )
gs3.povms['Mdefault'] = pygsti.obj.TensorProdPOVM( 
    [ pygsti.obj.TPPOVM([('0',v0),('1',v1)] ) for i in range(nQubits) ] )
Gx = std.gs_target['Gx'].copy()
Gy = std.gs_target['Gy'].copy()
for i in range(nQubits):
    gs3[('Gx','Q%d' % i)] = Gx
    gs3[('Gy','Q%d' % i)] = Gy

Gcnot = std2Q.gs_target['Gcnot'].copy()
gs3[('Gcnot','Q0','Q1')] = Gcnot
gs3[('Gcnot','Q1','Q2')] = Gcnot

gs3._calcClass = GateMapCalc
print(gs3.probs(c3))
print(c3.simulate(gs3))

In [ ]:
from pygsti.objects.gatemapcalc import GateMapCalc     
from pygsti.objects.labeldicts import StateSpaceLabels
nQubits = 3
basis1Q = pygsti.obj.Basis("pp",2)
v0 = pygsti.construction.basis_build_vector("0", basis1Q)
v1 = pygsti.construction.basis_build_vector("1", basis1Q)

gs2 = pygsti.obj.GateSet()
gs2.stateSpaceLabels = (0,1,2)
gs2.preps['rho0'] = pygsti.obj.TensorProdSPAMVec('prep',
    [pygsti.obj.TPParameterizedSPAMVec(v0) for i in range(nQubits)] )
gs2.povms['Mdefault'] = pygsti.obj.TensorProdPOVM( 
    [ pygsti.obj.TPPOVM([('0',v0),('1',v1)] ) for i in range(nQubits) ] )

Gx = std.gs_target['Gx'].copy()
Gy = std.gs_target['Gy'].copy()
for i in range(nQubits):
    gs2[('Gx',i)] = Gx
    gs2[('Gy',i)] = Gy

Gcnot = std2Q.gs_target['Gcnot'].copy()
gs2[('Gcnot',0,1)] = Gcnot
gs2[('Gcnot',1,2)] = Gcnot

gs2._calcClass = GateMapCalc
print(gs2.probs(c2))
print(c2.simulate(gs2))

## Unitary evolution in pyGSTi

In [ ]:
#Unitary gateset in pygsti
import scipy.linalg as spl
from pygsti.objects.gatematrixcalc import UnitaryGateMatrixCalc
from pygsti.objects.gatemapcalc import UnitaryGateMapCalc
X = np.array([[0.,1.],[1.,0.]])
Ut = np.array([[1.,0.],[0.,np.exp(np.pi*1j/4)]])
Ux = 1j*spl.expm( -1j* np.pi/4 * X) # pi/2
#Ux = np.array([[0.,1.],[1.,0.]]) # pi
v0 = np.array([[1],[0]],'d')
v1 = np.array([[0],[1]],'d')

gs = pygsti.obj.GateSet()
#gs._calcClass = UnitaryGateMatrixCalc
gs._calcClass = UnitaryGateMapCalc
gs['Gt'] = pygsti.obj.StaticGate(Ut)
gs['Gx'] = pygsti.obj.StaticGate(Ux)
gs['rho0'] = pygsti.obj.StaticSPAMVec( v0 )
gs['M0'] = pygsti.obj.UnconstrainedPOVM({'0': pygsti.obj.StaticSPAMVec( v0 ), '1': pygsti.obj.StaticSPAMVec( v1 )})
             

In [ ]:
Ux

In [ ]:
#print(gs.product(()))
#print()
print(gs.probs(()))

In [ ]:
#print(gs.product(('Gx',)))
#print()
print(gs.probs(('Gx',)))

In [ ]:
#print(gs.product(('Gt',)))
#print()
print(gs.probs(('Gt',)))

In [ ]:
#print(gs.product(('Gt','Gx')))
#print()
print(gs.probs(('Gt','Gx')))

In [ ]:
#print(gs.product(('Gx','Gx')))
#print()
print(gs.probs(('Gx','Gx')))

## Simulation of a "standard" n-qubit gate set
Based off of some of the initial cells in other test notebook.

In [ ]:
import pygsti
import numpy as np
# A list of standard hard-coded gate labels
#gllist = ['I', 'H','P','CNOT']
gllist = ['Gi','Gh','Gp','Gcnot','Gt'] # note: must include Gt here in addition to in 'unitaries' below

# A dictionary of unitaries that do not need to be already known to the code
#unitaries={'T' : np.array([[1.,0.],[0.,np.exp(np.pi*1j/4)]])}
unitaries={'Gt' : np.array([[1.,0.],[0.,np.exp(np.pi*1j/4)]])}

# The number of qubits
n = 10

# The availiability of gates that are not available to all qubit / qubit pairs
availability = {}

# Let's make a the CNOT gate be connected in a directed ring.
# a = np.zeros((n,n),int)
# for i in range(0,n-1):
#     a[i,i+1] = 1
# a[n-1,0] = 1
a = [(i,i+1) for i in range(0,n-1)] + [(n-1,0)]

#availability={'CNOT':a}
availability={'Gcnot':a}

BGS = pygsti.construction.build_nqubit_standard_gateset(
    nQubits=n, gate_names=gllist, nonstd_gate_unitaries=unitaries,
    availability=availability, parameterization='static', sim_type="dmmap")

In [ ]:
#print(BGS.preps['rho0'])
#print(list(map(str,BGS.gates.keys())))
#print(BGS)

In [ ]:
# test circuit simulation
circuit = pygsti.obj.Circuit(gatestring=( ('Gh',1),('Gp',2),('Gcnot',1,2),('Gt',3) ), num_lines=n)
%time print(BGS.probs(circuit))
# ~9 min for 10Q

## Processor Specs

In [ ]:
gllist = ['Gi', 'Gh','Gp','Gcnot','Gt']
ps = pygsti.obj.ProcessorSpec(nQubits=n, gate_names=gllist,
                              nonstd_gate_unitaries=unitaries,
                              availability=availability, verbosity=0)
  # automatically creates 'clifford' and 'target' models and std compilations
    
# To manually add another model, for example one w/density matrix simulation:
ps.models['dmsim'] = pygsti.construction.build_nqubit_standard_gateset(
    n, gllist, unitaries, availability, parameterization="static", sim_type="dmmap")
# OR, more conveniently
ps.add_std_model('dmsim', parameterization="static", sim_type="dmmap")

In [ ]:
print(ps.models['target']['Gcnot',0,1])

In [ ]:
ps.add_std_model('test', parameterization="static", sim_type="svmap")

In [ ]:
gs =  ps.models['test']

In [ ]:
print(gs['Gi',0])

In [ ]:
gs['Gi',0] = pygsti.objects.StaticGate(np.array([[1.,0.],[0.,1j]]))

In [ ]:
pygsti.tools.unitary_to_process_mx

In [ ]:
print(gs['Gi',0])

In [ ]:
#See where symplectic reps are stored (could also extract via gs.get_clifford_symplectic_reps() )
gs = ps.models['clifford']
print(gs[('Gcnot',0,1)].embedded_gate.smatrix) # each gate has it's own symplectic rep
assert(gs[('Gcnot',1,2)].embedded_gate is gs[('Gcnot',0,1)].embedded_gate) # different CNOT's share same "root"

In [ ]:
# Let's pick a random Clifford, and the inverse of that Clifford, and create an identity circuit
s, p = pygsti.tools.random_clifford(n)
sin, pin = pygsti.tools.inverse_clifford(s,p)

%time c = pygsti.alg.compile_clifford(s, p, pspec=ps)
%time c.append_circuit(pygsti.alg.compile_clifford(sin, pin, pspec=ps))
# ~3.5min each for 8Q, ~14min each for 10Q

print(len(c)) # 1389 for 8Q, 3057 for 10Q
print(c.depth()) # 1539 for 10Q

In [ ]:
# As we can see, the output is (0,0,0,0) with probability 1. (when no input is given to the simulators, the input
# is taken to be (0,0,0,0)).

%time simout = c.simulate(ps.models['target'])
# ~ 3.5s for 8Q, 47s fo 10Q

print(simout['0'*n])
x = 0
for key in list(simout.keys()):
    x += simout[key]
x = x - simout['0'*n]
print(x)